# KAKENの研究分野マスタをローカルのMariaDBに保存するプログラム

## 事前準備

- KAKENマスタデータは、git のリポジトリで管理されており、最新のデータを利用可能。
- https://bitbucket.org/niijp/grants_masterxml_kaken/ からリポジトリを pull して、ローカルの ./grants_masterxml_kaken フォルダに同期しておく。
- ローカルで MariaDB を動かしておく。
- MariaDB のユーザ名、パスワード、データベース名は、config.ipynbを使って ./config.ini として保存しておく。

## ここから本編

bitbucketから読み込んだマスタのXMLファイルをelementTreeに変換

In [ ]:
from lxml import etree
import pandas as pd
import numpy as np

tree = etree.parse('grants_masterxml_kaken/field_master_kakenhi.xml')

In [ ]:
fieldlist = []
for field_table in tree.iterfind("field_table"):
    field_table_type = field_table.get("type")
    field_table_start_date = field_table.get("start_date")
    field_table_end_date = field_table.get("end_date")
    
    for field in field_table.iterfind("field"):
        field_path = field.get("path")
        field_niicode = field.find("code[@type='nii']").text
        field_name = field.find("name[@lang='ja']").text
        row = [
            field_table_type,
            field_table_start_date,
            field_table_end_date,
            field_path,
            field_niicode,
            field_name,
            1,
        ]
        fieldlist.append(row)

        for field in field.iterfind("field"):
            field_path = field.get("path")
            field_niicode = field.find("code[@type='nii']").text
            field_name = field.find("name[@lang='ja']").text
            row = [
                field_table_type,
                field_table_start_date,
                field_table_end_date,
                field_path,
                field_niicode,
                field_name,
                2,
            ]
            fieldlist.append(row)
                
            for field in field.iterfind("field"):
                field_path = field.get("path")
                field_niicode = field.find("code[@type='nii']").text
                field_name = field.find("name[@lang='ja']").text
                row = [
                    field_table_type,
                    field_table_start_date,
                    field_table_end_date,
                    field_path,
                    field_niicode,
                    field_name,
                    3,
                ]
                fieldlist.append(row)

                for field in field.iterfind("field"):
                    field_path = field.get("path")
                    field_niicode = field.find("code[@type='nii']").text
                    field_name = field.find("name[@lang='ja']").text
                    row = [
                        field_table_type,
                        field_table_start_date,
                        field_table_end_date,
                        field_path,
                        field_niicode,
                        field_name,
                        4,
                    ]
                    fieldlist.append(row)
            
df = pd.DataFrame(fieldlist)
df.columns = [
    'field_table_type',
    'field_table_start_date',
    'field_table_end_date',
    'field_path',
    'field_niicode',
    'field_name',
    'layer',
]

重複のデータがあるかどうか

In [ ]:
df.duplicated().any()

重複のデータはなかった

In [ ]:
df['field_table_start_date'] = pd.to_datetime(df['field_table_start_date'])
df['field_table_end_date'] = pd.to_datetime(df['field_table_end_date'])

In [ ]:
df.info()

In [ ]:
df = df.astype({
    'field_niicode': int,
    'layer': int,
})

ローカルのmariaDBに関する設定ファイルを読み込みして、データベースにテーブル構造とデータの中身を書き込む

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('../settings/config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer, Date

engine = create_engine(url, echo=True)
df.to_sql('kaken_master_field', engine, if_exists='replace',
          dtype={
                'field_table_type' : String(256),
                'field_table_start_date' : Date,
                'field_table_end_date' : Date,
                'field_path' : String(256),
                'field_niicode' : Integer,
                'field_name' : String(256),
                'layer' : Integer,
          })

## おしまい

データがコミットされていれば終了。HeidiSQLなどで、上記ドロップ済みデータフレームの件数が登録されているか確認する。